In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
import mysql.connector as SQLC
import mysql.connector

### web scraping (escolher condição e nº de genes)

In [8]:
#escolha do query
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
#     print (a)

exist = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(exist)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: '))] #can be more. related to ncbi pagination
print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
# else:
#     print (n_genes)

escolha o que quer pesquisar: diabetes
escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: 20
['16847', '1493', '7422', '3569', '1029', '5468', '387082', '79068', '3630', '6934', '170801', '3481', '3784', '26191', '170802', '3559', '3667', '3172', '6927', '3375']


### Buscar Links dos genes:

In [9]:
# FINAL - necessário em pubmed (para script do rodrigo)
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format(id)
        url_list.append(url)
    return url_list
# url_get_id(Ids) 

### Buscar Links de proteinas na Uniprot:

In [10]:
#FINAL (usado em baixo) - NECESSÁRIO
def url_get_id_p(string):
#     print(string)
    url_list_p=[ ]
    url_id_p= "https://www.uniprot.org/uniprotkb/{}/entry".format(str(string))
    url_list_p.append(url_id_p)
    return ''.join(url_list_p)
print(url_get_id_p)

<function url_get_id_p at 0x000001E6E4F95550>


### Buscar ids do ncbi: Ids

In [11]:
# FINAL - Ids   (NECESSÁRIO)
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
print(Ids)

['Z17674.1', 'V00883.1', 'X67017.1', 'X61243.1', 'M90830.1', 'Z15032.1', 'V01310.1', 'M94340.1', 'X54964.1', 'Z19157.1', 'X05328.1']


### Buscar info para CDS: protein_id, location e product

In [12]:
#FINAL para protein_id - PROT_ID (NECESSÁRIO)
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

['CAA24252.1', 'CAA47407.1', 'CAA78750.1', 'CAA24617.1', 'CAA38712.1', 'CAB63418.2', 'CAE18055.3', 'CAA79568.2', 'CAA79570.2', 'CAA79569.3', 'CAA79566.3', 'CAA28945.1']


In [13]:
#FINAL result_dic = {Ids, ID_PROT} - NECESSÁRIO
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
# idlist= Ids.remove('U49845')
for ids in idlist:
    list_pro=[]
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A_CDS"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
print(result_dict)
#key: id_genebank; values: id_Uniprot

{'Z17674.1': 'N/A_CDS', 'V00883.1': 'CAA24252.1', 'X67017.1': 'CAA47407.1', 'X61243.1': 'N/A_CDS', 'M90830.1': 'N/A_CDS', 'Z15032.1': 'CAA78750.1', 'V01310.1': 'CAA24617.1', 'M94340.1': 'N/A_CDS', 'X54964.1': 'CAA38712.1', 'Z19157.1': 'CAB63418.2, CAE18055.3, CAA79568.2, CAA79570.2, CAA79569.3, CAA79566.3', 'X05328.1': 'CAA28945.1'}


In [14]:
#FINAL para location, product e translation  - get_CDS_info(result_dict) = [id_protein, location,translation] - NECESSÁRIO

def get_CDS_info(result_dict):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    cds_location_list = []
    several_location = []
    record_types={}
    for i, value in result_dict.items():
        cds_location =[]
        if value == 'N/A_CDS':
            cds_location.append(value)
            cds_location.append('N/A')            
            cds_location.append('N/A')                     
            cds_location_list.append(cds_location)

        else:
            handle = Entrez.efetch(db=database, id=i, rettype="gb") 
            records = list(SeqIO.parse(handle,"gb"))
            handle.close()
            for info in records:
                for i in info.features:
                    product, translation = '', ''
                    if i.type == "CDS":
                        i_d = str(i.qualifiers["protein_id"])
                        translation =  str(i.qualifiers["translation"])
                        cds_location.append(i_d)
                        if isinstance(i.location, CompoundLocation):
                            for sub_location in i.location.parts:
                                several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                            cds_location.append(several_location)
                            cds_location.append(translation)
                        else:
                            cds_location.append("[{} : {}]".format(i.location.start, i.location.end))
                            cds_location.append(translation)
                        cds_location_list.append(cds_location)      
                
            handle.close()
    return cds_location_list
print(get_CDS_info(result_dict))

[['N/A_CDS', 'N/A', 'N/A'], ["['CAA24252.1']", ['[223 : 316]', '[440 : 662]', '[1479 : 1608]', '[1077 : 1206]', '[1299 : 1359]', '[1460 : 1652]', '[1955 : 2632]', '[2996 : 3173]', '[3220 : 3314]', '[3557 : 3713]', '[3759 : 4226]', '[4474 : 4558]', '[4927 : 5093]', '[5604 : 5666]', '[5744 : 5837]', '[5885 : 6060]', '[11965 : 12069]', '[12179 : 12279]', '[10518 : 10605]', '[9659 : 10043]', '[9482 : 9611]', '[9157 : 9434]', '[8593 : 9110]', '[8461 : 8540]', '[8174 : 8408]', '[7810 : 8097]', '[7509 : 7723]', '[6491 : 7055]', '[6320 : 6440]', '[6211 : 6271]', '[18977 : 19035]', '[18483 : 18569]', '[18040 : 18433]', '[17645 : 17973]', '[17229 : 17598]', '[17059 : 17182]', '[16839 : 17013]', '[16317 : 16791]', '[15591 : 15953]', '[15410 : 15546]', '[15063 : 15361]', '[14610 : 14807]', '[14264 : 14561]', '[14026 : 14212]', '[13481 : 13976]', '[13171 : 13353]', '[12886 : 13124]', '[12746 : 12826]', '[25527 : 25691]', '[25244 : 25443]', '[25117 : 25190]', '[24440 : 24847]', '[23979 : 24232]', '[

In [15]:
#FINAL - NECESSÁRIO
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)
#############################BUG AQUI#########################################
def get_list_uniprot(ID_PROT, result_dict):
    results = {}
    result = {}
    for first_key, first_value in result_dict.items(): 
        result = {}
        for field in fields:
            if first_value != 'N/A_CDS':
                result[field] = get_field_for_id(first_value, field)
            else:
                result[field] = 'N/A_Uniprot'
            results[first_key] = result   
#############################BUG AQUI#########################################    
    uniprot_final_list=[]
    for key, value in results.items():
        uniprot_list=[]
        for x in value.values():
#             print(key)
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
            function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
            location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', x, re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'', x, re.DOTALL )   
            sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            n_a = re.search(r'(N/A_Uniprot)', x, re.DOTALL)
            if entry:
                ent=entry.group(1)
                uniprot_list.append(entry.group(1))            
            if function:
                uniprot_list.append(function.group(1))
#                 print(function.group(0))
#             if not function:
#                 print('na')
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append("N/A")
            if sequence:
                uniprot_list.append(sequence.group(1))
                uniprot_list.append(len(sequence.group(1)))
                uniprot_list.append(url_get_id_p(ent))
            if n_a:
                uniprot_list.append(n_a.group(1))
        uniprot_final_list.append(uniprot_list)
#     if len(uniprot_final_list)    
    return uniprot_final_list
get_Uniprot=get_list_uniprot(ID_PROT,result_dict) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)

In [22]:
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

def dict_to_list(d, delimiter=','):
    new_list = []
    for k, v in d.items():
        if isinstance(v, str):
            values = v.split(delimiter)
            for val in values:
                new_list.append([k, val.strip()])
        else:
            new_list.append([k, v])
    return new_list
dict_to_list(result_dict)

def get_list_uniprot(ID_PROT, result_dict):
    results = {}
    result = {}
    
    easy=dict_to_list(result_dict)
    
    for first_index, first_value in easy:         
        result = {}
        for field in fields:
            if first_value != 'N/A_CDS':
                result[field] = get_field_for_id(first_value, field)
            else:
                result[field] = 'N/A_Uniprot'
            results[first_index] = result   
    
    uniprot_final_list=[]
    for key, value in results.items():
        uniprot_list=[]
        for x in value.values():
#             print(key)
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
            function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
            location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', x, re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'', x, re.DOTALL )   
            sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            n_a = re.search(r'(N/A_Uniprot)', x, re.DOTALL)
            if entry:
                ent=entry.group(1)
                uniprot_list.append(entry.group(1))            
            if function:
                uniprot_list.append(function.group(1))
#                 print(function.group(0))
#             if not function:
#                 print('na')
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append("N/A")
            if sequence:
                uniprot_list.append(sequence.group(1))
                uniprot_list.append(len(sequence.group(1)))
                uniprot_list.append(url_get_id_p(ent))
            if n_a:
                uniprot_list.append(n_a.group(1))
        uniprot_final_list.append(uniprot_list)
#     if len(uniprot_final_list)    
    return uniprot_final_list
get_Uniprot=get_list_uniprot(ID_PROT,result_dict)

In [23]:
for i in get_Uniprot:
    print(i)

['N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot']
['P02099', 'N/A', 'This protein functions as an embryonic globin, but the gene structure and chromosomal location resemble more closely the human gamma chain gene, which codes for a fetal globin.', 'MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH', 147, 'https://www.uniprot.org/uniprotkb/P02099/entry']
['Q00873', 'Mitochondrion inner membrane {ECO:0000269|PubMed:1499554}. Note=Partially exposed to the intermembrane space. {ECO:0000305|PubMed:1499554}.', 'Lyase that catalyzes the covalent linking of the heme group to the cytochrome C1 apoprotein to produce the mature functional cytochrome. {ECO:0000269|PubMed:1499554}.', 'MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSANPQDNDKTPEYHTTVDLSQSREVSTIPRTNSDRNWIYPSEKQFYEAMMKKNWDPNSDDMKVVVPLHNSINERVWNYIKSWEDKQGGEACGGIKLTNFKGDSKKLTPRAWFRSRILHLAKPFDRHDWQIDRCGKTVDY

In [18]:
def dict_to_list(d, delimiter=','):
    new_list = []
    for k, v in d.items():
        if isinstance(v, str):
            values = v.split(delimiter)
            for val in values:
                new_list.append([k, val.strip()])
        else:
            new_list.append([k, v])
    return new_list
easy=dict_to_list(result_dict)

In [21]:
for first_index, first_value in easy:
    print(first_value)

N/A_CDS
CAA24252.1
CAA47407.1
N/A_CDS
N/A_CDS
CAA78750.1
CAA24617.1
N/A_CDS
CAA38712.1
CAB63418.2
CAE18055.3
CAA79568.2
CAA79570.2
CAA79569.3
CAA79566.3
CAA28945.1


### Buscar Links de proteinas:

In [25]:
#final, mas não correr (código necessário para o caso se querermos links não diretos para uniprot )
def url_get_id_p(dic):
    url_list_p = []
    for key, value in dic.items():
        if value != 'N/A_CDS':
            value_list = value.split(',')
            for val in value_list:
                val = val.strip()
                url_id_p = "https://www.uniprot.org/uniprotkb?query={}".format(val)
                url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(result_dict)
print(url_ids_protein) # but we don't need to save this links

['https://www.uniprot.org/uniprotkb?query=CAA27540.1', 'https://www.uniprot.org/uniprotkb?query=AAA47844.1']


In [71]:
#FINAL [idgenebank, protein_id, id do uniprot]  -  NECESSÁRIO
def join_ids_CDS(dic, uniprotID):
    join_list_all = []
    for key, value in dic.items():
        join_list = []
        join_list.append(key)
        join_list.append(value)
        join_list_all.append(join_list) 
    for index, values in enumerate(uniprotID):
        join_list_all[index].append(uniprotID[index][0])
    return join_list_all
join_ids=join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))

In [73]:
#final [idgenebank, protein_id, id_uniprot,location, product, translation] (NÃO SEI SE ESTÁ A FUNCAR DIREITO - assumir que não)
def join_lists(list1, list2):
    final_result =[]
    
    for item, value in enumerate(list1):
        result = []
        string_item = str(list2[item][0])
        string_item = string_item.replace("[","").replace("]","").replace("'","")
        if list1[item][1] == string_item:
            result.append(list1[item] + list2[item][1:])
        else:
            result.append(item1[item])
        final_result.append(result)
    return final_result
join_CDS=join_lists(join_ids, get_CDS_info(result_dict))

In [79]:
for a in join_CDS:
    for b in a:
        print(b[0])

X03907.1
AA928418.1
AAADR0035516.1
L20655.1
AAADU0045050.1
AAAEJ0028203.1
AAAEJ0013167.1
F00426.1
AA628858.1
T76582.1


In [59]:
def count_(genes):
    return len(genes)
count_(Ids)
count_(ID_PROT)

2

### Começar povoar

In [18]:
# TESTES para povoar CDS
# print(result_dict)
# print(Ids)
# print(get_CDS_location(Ids)[0])
#key: id_genebank values: id_Uniprot
for i in get_CDS_info(Ids):
    print(i)

['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
["['CAA24252.1']", ['[223 : 316]', '[440 : 662]', '[1479 : 1608]'], "['beta-3 globin']", "['MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH']"]
["['CAA47407.1']", '[246 : 921]', "['cytochrome c1 heme lyase']", "['MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSANPQDNDKTPEYHTTVDLSQSREVSTIPRTNSDRNWIYPSEKQFYEAMMKKNWDPNSDDMKVVVPLHNSINERVWNYIKSWEDKQGGEACGGIKLTNFKGDSKKLTPRAWFRSRILHLAKPFDRHDWQIDRCGKTVDYVIDFYSTDLNDANSQQQPLIYLDVRPKLNSFEGFRLRFWKSLGF']"]


In [94]:
for i in get_Uniprot:
    print(i)

['P0CJ48', 'Plastid, chloroplast thylakoid membrane; Multi-pass membrane protein.', 'The light-harvesting complex (LHC) functions as a light receptor, it captures and delivers excitation energy to photosystems with which it is closely associated.', 'MAASTMALSSPAFAGKAVNLSPAASEVLGSGRVTMRKTVAKPKGPSGSPWYGSDRVKYLGPFSGESPSYLTGEFPGDYGWDTAGLSADPETFARNRELEVIHSRWAMLGALGCVFPELLARNGVKFGEAVWFKAGSQIFSDGGLDYLGNPSLVHAQSILAIWATQVILMGAVEGYRVAGNGPLGEAEDLLYPGGSFDPLGLATDPEAFAELKVKELKNGRLAMFSMFGFFVQAIVTGKGPIENLADHLADPVNNNAWAFATNFVPGK', 267, 'https://www.uniprot.org/uniprotkb/P0CJ48/entry']
['N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot']
['N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot']
['Q88364', 'N/A', 'LISEAETRGITGYNPLAGPLRVQANNPQQQG', 31, 'https://www.uniprot.org/uniprotkb/Q88364/entry']
['N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot']
['N/A_Uniprot', 'N/A_Uniprot', 'N/A_Uniprot',

In [93]:
#Povoação "Uniprot" 
# print(get_Uniprot) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa)

DataBase = SQLC.connect(
    host ="localhost",
    user ="root",
    password ="Punzel18",
    database ="ap_db_krg"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
#     sql = "ALTER TABLE Uniprot AUTO_INCREMENT = 0"
#     Cursor.execute(sql)
    for index in get_Uniprot:
        #print(value)
        sql= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(str(index[0]), str(index[1]), str(index[2]), str(index[3]), str(index[4]), str(index[5]))
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

Erro na escrita na base de dados: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Function, Protein_sequence, length_aa, Link_Uniprot) VALUES ('P0CJ48', 'Plastid,' at line 1


In [84]:
#Povoação "CDS"

DataBase = SQLC.connect(
    host ="localhost",
    user ="root",
    password ="Punzel18",
    database ="ap_db_krg"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index in join_CDS:
        for item in index:
            sql= "INSERT INTO CDS (ID_CDS, Translation, Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s)"
            val=(item[1],item[4],item[3],item[0],item[2])
            Cursor.execute(sql,val)
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

Erro na escrita na base de dados: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`ap_db_krg`.`cds`, CONSTRAINT `uniprot` FOREIGN KEY (`ID_Uniprot`) REFERENCES `uniprot` (`ID_Uniprot`))


In [107]:
#TESTES IMPORTANTES PARA DUPLICADOS
AA=[["BA","NA","NA","NA","NA","NA"],["N/A","N/A","N/A","N/A","N/A","N/A"]]
a=["N/A"]
DataBase = SQLC.connect(
    host ="localhost",
    user ="root",
    password ="Punzel18",
    database ="ap_db_krg"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sec = 'INSERT INTO uniprot (ID_Uniprot) VALUES (%s)'
    Cursor.execute(sec, a)
    for i, a in enumerate(AA):
        if AA[i][0] == 'N/A':
            continue
        else:
            sql= 'INSERT INTO uniprot (ID_Uniprot, Subcelular_Location, function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)'
            val=(AA[i][0],AA[i][1],AA[i][2],AA[i][3],AA[i][4],AA[i][5])
            Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

Erro na escrita na base de dados: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'function, Protein_sequence, length_aa, Link_Uniprot) VALUES ('BA', 'NA', 'NA', '' at line 1


In [106]:
DataBase = SQLC.connect(
    host ="localhost",
    user ="root",
    password ="Punzel18",
    database ="ap_db_krg"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
#     sql = "ALTER TABLE Uniprot AUTO_INCREMENT = 0"
#     Cursor.execute(sql)
    sql= 'delete from uniprot'
    Cursor.execute(sql)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [85]:
import pandas as pd 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)

####### end of connection ####
tabela = pd.read_sql("Select * FROM Gene", DataBase)
DataBase.close()
print(tabela)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'geo.di.uminho.pt:3306' (10060)

In [57]:
import pandas as pd 
import mysql.connector as SQLC
import mysql.connector

DataBase = SQLC.connect(
    host ="localhost",
    user ="root",
    password ="Punzel18",
    database ="ap_db_krg"
)

####### end of connection ####
tabela = pd.read_sql("Select * FROM uniprot", DataBase)
DataBase.close()
print(tabela)

Empty DataFrame
Columns: [ID_Uniprot, Function, Subcelular_Location, Protein_sequence, length_aa, Link_Uniprot]
Index: []


C:\Users\Karyna\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
